<a href="https://colab.research.google.com/github/okeefey/MIDIGenerator/blob/main/MIDIGenerator_using_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [3]:

from music21 import *
import glob
from tqdm import tqdm
import numpy as np
import random
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout
from tensorflow.keras.models import Sequential,Model,load_model
from sklearn.model_selection import train_test_split



# Reading MIDI files


# Reading notes

In [4]:
from google.colab import drive
drive.mount('/content/drive')

def read_files(file):
    notes = []
    try:
        midi = converter.parse(file)
        notes_to_parse = midi.flat.getElementsByClass([note.Note, chord.Chord])

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                chord_notes = '.'.join(str(pitch) for pitch in element.pitches)
                notes.append(chord_notes)
    except Exception as e:
        print(f"Error processing {file}: {e}")
    return notes

all_files = glob.glob('/content/drive/MyDrive/All Midi Files/**/*.mid', recursive=True)
print(f"Found {len(all_files)} files.")

notes_list = [read_files(file) for file in tqdm(all_files, position=0, leave=True)]

Mounted at /content/drive
Found 93 files.


  0%|          | 0/93 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/music21/stream/base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
100%|██████████| 93/93 [00:30<00:00,  3.01it/s]


In [5]:

notess = sum(notes_list, [])  # Flatten the list of lists
unique_notes = list(set(notess))
print("Unique Notes:", len(unique_notes))

# Calculate frequency using Counter
from collections import Counter
freq = Counter(notess)

for i in range(10, 50, 10):
    count = sum(1 for cnt in freq.values() if cnt >= i)

Unique Notes: 557


### Threshold

In [6]:

freq_notes=dict(filter(lambda x:x[1]>=5,freq.items()))


new_notes=[[i for i in j if i in freq_notes] for j in notes_list]

In [7]:
ind2note=dict(enumerate(freq_notes))

note2ind=dict(map(reversed,ind2note.items()))

# Timestep

#### Every x notes will output one note

In [8]:

timesteps=50


x=[] ; y=[]

for i in new_notes:
 for j in range(0,len(i)-timesteps):
  #input will be the current index + timestep
  #output will be the next index after timestep
  inp=i[j:j+timesteps] ; out=i[j+timesteps]

  #append the index value of respective notes
  x.append(list(map(lambda x:note2ind[x],inp)))
  y.append(note2ind[out])

x_new=np.array(x)
y_new=np.array(y)

In [9]:
x_new = np.reshape(x_new,(len(x_new),timesteps,1))
y_new = np.reshape(y_new,(-1,1))

#80% for training and 20% for testing sets
x_train,x_test,y_train,y_test = train_test_split(x_new,y_new,test_size=0.2,random_state=42)


In [10]:
model = Sequential()
model.add(LSTM(128,return_sequences=True,input_shape=(x_new.shape[1],x_new.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(Dense(len(note2ind),activation='softmax'))
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50, 128)        │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 65)             │         8,385 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 223,041 (871.25 KB)

 Trainable params: 223,041 (871.25 KB)

 Non-trainable params: 0 (0.00 B)

# Model Training

In [11]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

model.fit(
 x_train,y_train,
 batch_size=128,epochs=120,
 validation_data=(x_test,y_test))

Epoch 1/120
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.0130 - loss: 4.1988 - val_accuracy: 0.1000 - val_loss: 4.1304
Epoch 2/120
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 575ms/step - accuracy: 0.1169 - loss: 4.0652 - val_accuracy: 0.1000 - val_loss: 4.0691
Epoch 3/120
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step - accuracy: 0.0909 - loss: 3.9427 - val_accuracy: 0.1000 - val_loss: 3.9970
Epoch 4/120
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 608ms/step - accuracy: 0.1429 - loss: 3.8358 - val_accuracy: 0.1000 - val_loss: 3.9169
Epoch 5/120
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 563ms/step - accuracy: 0.1429 - loss: 3.6995 - val_accuracy: 0.1000 - val_loss: 3.8423
Epoch 6/120
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step - accuracy: 0.1429 - loss: 3.5653 - val_accuracy: 0.1000 - val_loss: 3.7731
Epoch 7/120
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 617ms/step - accuracy: 0.1299 - loss: 3.4104 - val_accuracy: 0.1000 - val_loss: 3.7150
Epoch 8/120
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 412ms/step - accuracy: 0.1299 - loss: 3.2996 - val_accuracy: 0.1000 - val_

# Save the data

In [12]:
model.save("model.keras")

# Composition

In [13]:
model = load_model("model.keras")
index = np.random.randint(0,len(x_test)-1)
music_pattern = x_test[index]
out_pred=[]

for i in range(50):


 music_pattern = music_pattern.reshape(1,len(music_pattern),1)

 pred_index = np.argmax(model.predict(music_pattern))

 out_pred.append(ind2note[pred_index])
 music_pattern = np.append(music_pattern,pred_index)


 music_pattern = music_pattern[1:]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 430ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━

# Saving the midi file

In [14]:
output_notes = []

print("Predicted Notes (out_pred):", out_pred)

try:
    for offset, pattern in enumerate(out_pred):

        if '.' in pattern or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            parsed_notes = []


            for current_note in notes_in_chord:
                if current_note.isdigit():
                    parsed_note = note.Note(int(current_note))
                else:
                    parsed_note = note.Note(current_note)
                parsed_notes.append(parsed_note)

            parsed_notes.sort(key=lambda x: x.pitch.midi)
            bass_note = parsed_notes[0]

            bass_note.storedInstrument = instrument.Piano()
            bass_note.duration = duration.Duration(10.0)
            bass_note.offset = offset
            output_notes.append(bass_note)

            remaining_notes = parsed_notes[1:]
            if remaining_notes:
                higher_chord = chord.Chord(remaining_notes)
                higher_chord.storedInstrument = instrument.Piano()
                higher_chord.duration = duration.Duration(1.0)  # Quarter note
                higher_chord.offset = offset
                output_notes.append(higher_chord)

        else:
            if pattern.isdigit():
                new_note = note.Note(int(pattern))
            else:
                new_note = note.Note(pattern)

            # Treat single notes as bass notes with long duration
            new_note.storedInstrument = instrument.Piano()
            new_note.duration = duration.Duration(4.0)  # Whole note
            new_note.offset = offset
            output_notes.append(new_note)

except Exception as e:
    print(f"Error during note creation: {e}")

print("Generated Notes (output_notes):", output_notes)

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='/content/drive/MyDrive/test_output.mid')

# For Google Colab, download the file
from google.colab import files
files.download('/content/drive/MyDrive/test_output.mid')

Predicted Notes (out_pred): ['F5.B-2', 'C#5', 'E-5', 'E-5', 'F5', 'G#5', 'G#5', 'G#5', 'F5', 'E-5.F5', 'E-5.F5', 'E-5', 'C#5', 'E-3', 'C5', 'C5', 'G#3', 'G#4', 'B-4', 'C5', 'C#5.G#3', 'E-5', 'F5.B-2', 'F5.B-2', 'C#5', 'E-5', 'E-5', 'F5', 'G#5', 'G#5', 'G#5', 'G#5', 'F5', 'E-5.F5', 'C#5', 'E-3', 'E-3', 'C5', 'C5', 'G#3', 'G#4', 'B-4', 'C5', 'C#5.G#3', 'E-5', 'F5.B-2', 'F5.B-2', 'C#5', 'E-5', 'E-5']
Generated Notes (output_notes): [<music21.note.Note B->, <music21.chord.Chord F5>, <music21.note.Note C#>, <music21.note.Note E->, <music21.note.Note E->, <music21.note.Note F>, <music21.note.Note G#>, <music21.note.Note G#>, <music21.note.Note G#>, <music21.note.Note F>, <music21.note.Note E->, <music21.chord.Chord F5>, <music21.note.Note E->, <music21.chord.Chord F5>, <music21.note.Note E->, <music21.note.Note C#>, <music21.note.Note E->, <music21.note.Note C>, <music21.note.Note C>, <music21.note.Note G#>, <music21.note.Note G#>, <music21.note.Note B->, <music21.note.Note C>, <music21.note

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>